# Agent Architecture

##### Reflection

In [1]:
from typing import Annotated, TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages

In [2]:
# Initialize chat model
model = ChatOpenAI()


# Define state type
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]


# Define prompts
generate_prompt = SystemMessage(
    "You are an essay assistant tasked with writing excellent 3-paragraph essays."
    " Generate the best essay possible for the user's request."
    " If the user provides critique, respond with a revised version of your previous attempts."
)

reflection_prompt = SystemMessage(
    "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
    " Provide detailed recommendations, including requests for length, depth, style, etc."
)

In [3]:
def generate(state: State) -> State:
    answer = model.invoke([generate_prompt] + state["messages"])
    return {"messages": [answer]}


def reflect(state: State) -> State:
    # Invert the messages to get the LLM to reflect on its own output
    cls_map = {AIMessage: HumanMessage, HumanMessage: AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [reflection_prompt, state["messages"][0]] + [
        cls_map[msg.__class__](content=msg.content) for msg in state["messages"][1:]
    ]
    answer = model.invoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=answer.content)]}


def should_continue(state: State):
    if len(state["messages"]) > 6:
        # End after 3 iterations, each with 2 messages
        return END
    else:
        return "reflect"

In [4]:
# Build the graph
builder = StateGraph(State)
builder.add_node("generate", generate)
builder.add_node("reflect", reflect)
builder.add_edge(START, "generate")
builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")

graph = builder.compile()

In [6]:
# Example usage
initial_state = {
    "messages": [
        HumanMessage(
            content="오늘날 '어린 왕자'의 관련성에 대한 에세이를 작성해 보세요."
        )
    ]
}

# Run the graph
for output in graph.stream(initial_state):
    message_type = "generate" if "generate" in output else "reflect"
    print("\nNew message:", output[message_type]
          ["messages"][-1].content[:100], "...")


New message: '어린 왕자'라는 책은 세계적으로 사랑받는 문학 작품으로, 여전히 현재에도 그 안에 담긴 메시지와 가치가 우리에게 큰 영감을 주고 있습니다. 이 소설은 어른과 아이, 이성과 감성, ...

New message: 이 에세이는 '어린 왕자'의 현대적인 관련성을 잘 소개하고 간략히 설명하고 있습니다. 그러나 조금 더 심도 있는 분석과 구체적인 예시를 통해 의견을 뒷받침할 수 있도록 노력해야 합 ...

New message: '어린 왕자'는 현대 사회에서 여전히 깊은 감정과 생각을 일으키는 문학적 걸작으로 손꼽힙니다. 이 작품은 무엇보다도 소소한 순간과 순수한 감수성에 대한 사색을 통해 현대 사회의 고 ...

New message: 이번에 작성한 에세이는 '어린 왕자'의 현대적인 의미와 가치를 잘 포착하고 있습니다. 그러나 더 깊이 있는 분석과 다양한 예시를 통해 주장을 보다 강화하고 전달력을 높일 수 있도록 ...

New message: '어린 왕자'는 현대 사회에도 여전히 많은 의미와 가치를 담고 있는 문학 작품입니다. 이 작품은 어린 왕자와 여우, 그리고 장미와 별들의 대화를 통해 인간의 본질과 삶의 의미를 탐 ...

New message: 이번에 작성한 에세이는 '어린 왕자'의 현대적인 가치와 의미를 잘 강조하고 있습니다. 그러나 좀 더 다양한 측면을 포함하고 깊이 있는 분석을 통해 주제를 보다 포괄적으로 다룰 수  ...

New message: '어린 왕자'는 현대 사회에서도 여전히 빛을 발하는 문학적 명작으로, 그 안에 담긴 다양한 주제와 메시지들이 오늘날의 우리에게도 큰 울림을 전달하고 있습니다. 이 작품은 자아성찰과 ...


##### Subgraph Direct 

In [7]:
from typing import TypedDict
from langgraph.graph import START, StateGraph


# Define the state types for parent and subgraph
class State(TypedDict):
    foo: str  # this key is shared with the subgraph


class SubgraphState(TypedDict):
    foo: str  # this key is shared with the parent graph
    bar: str

In [ ]:
# Define subgraph
def subgraph_node(state: SubgraphState):
    # note that this subgraph node can communicate with the parent graph via the shared "foo" key
    return {"foo": state["foo"] + "bar"}


subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node("subgraph_node", subgraph_node)
# Additional subgraph setup would go here
# subgraph_builder.add_edge(START, "subgraph_node")
subgraph = subgraph_builder.compile()

# Define parent graph
builder = StateGraph(State)
builder.add_node("subgraph", subgraph)
builder.add_edge(START, "subgraph")
# Additional parent graph setup would go here
graph = builder.compile()

In [11]:
# Example usage
initial_state = {"foo": "hello"}
result = graph.invoke(initial_state)
print(f"Result: {result}")  # Should append "bar" to the foo value

Result: {'foo': 'hellobar'}


##### Subgraph Function

In [12]:
from typing import TypedDict
from langgraph.graph import START, StateGraph


class State(TypedDict):
    foo: str


class SubgraphState(TypedDict):
    # none of these keys are shared with the parent graph state
    bar: str
    baz: str

In [13]:
# Define subgraph
def subgraph_node(state: SubgraphState):
    return {"bar": state["bar"] + "baz"}


subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node("subgraph_node", subgraph_node)
subgraph_builder.add_edge(START, "subgraph_node")
# Additional subgraph setup would go here
subgraph = subgraph_builder.compile()


# Define parent graph node that invokes subgraph
def node(state: State):
    # transform the state to the subgraph state
    response = subgraph.invoke({"bar": state["foo"]})
    # transform response back to the parent state
    return {"foo": response["bar"]}


builder = StateGraph(State)
# note that we are using `node` function instead of a compiled subgraph
builder.add_node("node", node)
builder.add_edge(START, "node")
# Additional parent graph setup would go here
graph = builder.compile()

In [14]:
# Example usage
initial_state = {"foo": "hello"}
result = graph.invoke(initial_state)
print(
    f"Result: {result}"
)  # Should transform foo->bar, append "baz", then transform bar->foo


Result: {'foo': 'hellobaz'}


##### Supervisor

In [45]:
from typing import Literal
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START
from langchain_core.messages import AIMessage, HumanMessage

# ──────────────────────────────
# 1) 모델 준비
# ──────────────────────────────
class SupervisorDecision(BaseModel):
    next: Literal["researcher", "coder", "FINISH"]

decision_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) \
               .with_structured_output(SupervisorDecision)   # supervisor 전용
agent_llm    = ChatOpenAI(model="gpt-4o-mini", temperature=0) # 일반 대화용

# ──────────────────────────────
# 2) 프롬프트와 노드 정의
# ──────────────────────────────
agents = ["researcher", "coder"]
sys1 = f"""You are a supervisor managing: {agents}.
Choose *one* worker to act next. When everything is done, answer FINISH."""
sys2 = f"Who should act next? Choose from {', '.join(agents)}, FINISH."

class AgentState(MessagesState):
    next: Literal["researcher", "coder", "FINISH"]

def supervisor(state: AgentState):
    messages = [("system", sys1), *state["messages"], ("system", sys2)]
    decision: SupervisorDecision = decision_llm.invoke(messages)
    # supervisor가 말도 남기고 싶다면 메시지로 감싼다
    
    # 콘솔에 supervisor의 판단을 출력
    print("\n📋 [Supervisor Decision]")
    print(f"- Available agents: {', '.join(agents)}")
    print(f"- User request: {state['messages'][-1].content}")
    print(f"- 💡 Decision: {decision.next}")
    
    return {
        "next": decision.next,
        "messages": [AIMessage(content=f"[Supervisor] next → {decision.next}")]
    }

def researcher(state: AgentState):
    prompt = [
        ("system", "You are a research assistant."),
        ("user",   state["messages"][-1].content),
    ]
    reply = agent_llm.invoke(prompt)
    return {"messages": [reply]}   # reply는 AIMessage

def coder(state: AgentState):
    prompt = [
        ("system", "You are a coding assistant."),
        ("user",   state["messages"][-1].content),
    ]
    reply = agent_llm.invoke(prompt)
    return {"messages": [reply]}

# ──────────────────────────────
# 3) 그래프 빌드
# ──────────────────────────────
builder = StateGraph(AgentState)
builder.add_node("supervisor", supervisor)
builder.add_node("researcher", researcher)
builder.add_node("coder", coder)

builder.add_edge(START, "supervisor")
builder.add_conditional_edges("supervisor", lambda s: s["next"])
builder.add_edge("researcher", "supervisor")
builder.add_edge("coder",      "supervisor")
graph = builder.compile()

# ──────────────────────────────
# 4) 실행
# ──────────────────────────────
initial_state = {
    "messages": [HumanMessage(content="I need help analysing data and creating a visualization.")],
    "next": "supervisor"
}

for step in graph.stream(initial_state):
    print(f"\n[STEP] next = {step.get('next')}")
    if step.get("messages"):
        print(step["messages"][-1].content)



📋 [Supervisor Decision]
- Available agents: researcher, coder
- User request: I need help analysing data and creating a visualization.
- 💡 Decision: researcher

[STEP] next = None

[STEP] next = None

📋 [Supervisor Decision]
- Available agents: researcher, coder
- User request: It seems like you're looking for a way to transition from a supervisor role to a researcher role. Here are some steps you might consider:

1. **Identify Your Research Interests**: Determine what areas of research excite you the most. This could be related to your current field or a new area you want to explore.

2. **Enhance Your Skills**: Depending on your research interests, you may need to acquire new skills or knowledge. This could involve taking courses, attending workshops, or self-study.

3. **Network with Researchers**: Connect with researchers in your field of interest. Attend conferences, seminars, and workshops to meet professionals and learn about current research trends.

4. **Seek Research Opportu